In [1]:
import blur
import hico_batch_scraper as hs
from http.cookiejar import CookieJar
import urllib

ModuleNotFoundError: No module named 'cv2'

In [ ]:
import os

folder_name = "hico_data"

if not os.path.exists(folder_name):
    os.makedirs(folder_name)


In [ ]:

parser = hs.MyHTMLParser() 

#===============================================================================
# The following code block is used for HTTPS authentication
#===============================================================================

# The user credentials that will be used to authenticate access to the data
username = "kimado"
password = "&82m%wPhE8-9.C+"

# The FULL url of the directory which contains the files you would like to bulk download
url = "https://oceandata.sci.gsfc.nasa.gov/directdataaccess/Level-1B/HICO/2014" # Example URL

# Create a password manager to deal with the 401 reponse that is returned from
# Earthdata Login
 
password_manager = urllib.request.HTTPPasswordMgrWithDefaultRealm()
password_manager.add_password(None, "https://urs.earthdata.nasa.gov", username, password)
 
# Create a cookie jar for storing cookies. This is used to store and return
# the session cookie given to use by the data server (otherwise it will just
# keep sending us back to Earthdata Login to authenticate).  Ideally, we
# should use a file based cookie jar to preserve cookies between runs. This
# will make it much more efficient.
 
cookie_jar = CookieJar()

# Install all the handlers.
opener = urllib.request.build_opener(
    urllib.request.HTTPBasicAuthHandler(password_manager),
    #urllib.request.HTTPHandler(debuglevel=1),    # Uncomment these two lines to see
    #urllib.request.HTTPSHandler(debuglevel=1),   # details of the requests/responses
    urllib.request.HTTPCookieProcessor(cookie_jar))
urllib.request.install_opener(opener)
 
# Create and submit the requests. There are a wide range of exceptions that
# can be thrown here, including HTTPError and URLError. These should be
# caught and handled.

#===============================================================================
# Open a requeset to grab filenames within a directory. Print optional
#===============================================================================

# Get the subdirectories
DirRequest = urllib.request.Request(url)
DirResponse = urllib.request.urlopen(DirRequest)
DirBody = DirResponse.read()
parser.feed(str(DirBody))
subdirs = parser.dataList

# Get the number of subdirectories to download from
x = 1 # Number of subdirectories to download from

# Download the files from the top 'x' subdirectories
for subdir in subdirs[:x]:
    # Construct the URL of the subdirectory
    subdir_url = url + subdir

    # Get the files in the subdirectory
    DirRequest = urllib.request.Request(subdir_url)
    DirResponse = urllib.request.urlopen(DirRequest)
    DirBody = DirResponse.read()
    parser.feed(str(DirBody))
    files = parser.dataList

    #===============================================================================
    # Call the function to download all files in url
    #===============================================================================

    # Download each file
    hs.BatchJob(files, cookie_jar, subdir_url, folder_name)

    # Reset the dataList for the next subdirectory
    parser.dataList = []

# Display the contents of the python list declared in the HTMLParser class
# print Files #Uncomment to print a list of the files

